In [1]:
import boto3
import json

In [2]:
session = boto3.Session()
sts_client = session.client('sts')
identity = sts_client.get_caller_identity()
print(identity['Arn'])

arn:aws:sts::794038231401:assumed-role/SageMaker-ExecutionRole-20250401T103257/SageMaker


In [3]:
bedrock = boto3.client(service_name='bedrock', region_name="us-east-1")
bedrock_runtime = boto3.client(service_name='bedrock-runtime', region_name="us-east-1")

In [4]:
# bedrock.list_foundation_models()
# print the list of foundation models just using the name
for model in bedrock.list_foundation_models()['modelSummaries']:
    print(model['modelId'])

amazon.titan-tg1-large
amazon.titan-image-generator-v1:0
amazon.titan-image-generator-v1
amazon.titan-image-generator-v2:0
amazon.nova-premier-v1:0:8k
amazon.nova-premier-v1:0:20k
amazon.nova-premier-v1:0:1000k
amazon.nova-premier-v1:0:mm
amazon.nova-premier-v1:0
amazon.titan-text-premier-v1:0
amazon.nova-pro-v1:0:24k
amazon.nova-pro-v1:0:300k
amazon.nova-pro-v1:0
amazon.nova-lite-v1:0:24k
amazon.nova-lite-v1:0:300k
amazon.nova-lite-v1:0
amazon.nova-canvas-v1:0
amazon.nova-reel-v1:0
amazon.nova-reel-v1:1
amazon.nova-micro-v1:0:24k
amazon.nova-micro-v1:0:128k
amazon.nova-micro-v1:0
amazon.nova-sonic-v1:0
amazon.titan-embed-g1-text-02
amazon.titan-text-lite-v1:0:4k
amazon.titan-text-lite-v1
amazon.titan-text-express-v1:0:8k
amazon.titan-text-express-v1
amazon.titan-embed-text-v1:2:8k
amazon.titan-embed-text-v1
amazon.titan-embed-text-v2:0:8k
amazon.titan-embed-text-v2:0
amazon.titan-embed-image-v1:0
amazon.titan-embed-image-v1
stability.stable-diffusion-xl-v1:0
stability.stable-diffusion

In [5]:
# Define the model ID for Meta Llama 3.2 11B Instruct
model_id = 'meta.llama3-2-11b-instruct-v1:0'

# Define the user prompt
user_prompt = "What is the capital of France?"

# Format the prompt according to Meta Llama's requirements
prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|> {user_prompt} <|eot_id|><|start_header_id|>assistant<|end_header_id|>"

# Define the request body with the prompt and optional parameters
body = json.dumps({
    "prompt": prompt,
    "temperature": 0.5,  # Controls randomness (default is 0.5)
    "max_gen_len": 512   # Maximum length of the generated response (default is 512)
})

In [9]:
# Invoke the model
response = bedrock_runtime.invoke_model(
    modelId=model_id,
    body=body,
    accept="application/json",
    contentType="application/json"
)

# Parse and print the response
response_body = json.loads(response['body'].read().decode('utf-8'))
print(response_body)

AccessDeniedException: An error occurred (AccessDeniedException) when calling the InvokeModel operation: User: arn:aws:sts::794038231401:assumed-role/SageMaker-ExecutionRole-20250401T103257/SageMaker is not authorized to perform: bedrock:InvokeModel on resource: arn:aws:bedrock:us-east-1:794038231401:inference-profile/us.meta.llama3-2-11b-instruct-v1:0 because no identity-based policy allows the bedrock:InvokeModel action

### so the model is not supported without cross region inference. Hence, we use an inference profile

### we attach the following policy:

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:*"
            ],
            "Resource": [
                "arn:aws:bedrock:us-east-1:794038231401:inference-profile/us.meta.llama3-2-11b-instruct-v1:0"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:*"
            ],
            "Resource": [
                "arn:aws:bedrock:us-east-1::foundation-model/meta.llama3-2-11b-instruct-v1:0",
                "arn:aws:bedrock:us-west-2::foundation-model/meta.llama3-2-11b-instruct-v1:0"
            ],
            "Condition": {
                "StringLike": {
                    "bedrock:InferenceProfileArn": "arn:aws:bedrock:us-east-1:794038231401:inference-profile/us.meta.llama3-2-11b-instruct-v1:0"
                }
            }
        }
    ]
}
```

In [17]:
# Define the model ID for Meta Llama 3.2 11B Instruct
model_id = 'us.meta.llama3-2-11b-instruct-v1:0'

# Invoke the model
response = bedrock_runtime.invoke_model(
    modelId=model_id,
    body=body,
    accept="application/json",
    contentType="application/json"
)

# Parse and print the response
response_body = json.loads(response['body'].read().decode('utf-8'))
print(response_body)

{'generation': '\n\nThe capital of France is Paris.', 'prompt_token_count': 16, 'generation_token_count': 9, 'stop_reason': 'stop'}
